In [38]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd
import json
from pandas import json_normalize
import re

file_path = './finished/'
file_name = 'war20240914.json'

full_path = os.path.join(file_path, file_name)

with open (full_path, 'r') as json_file:
    response = json.load(json_file)
response

{'state': 'warEnded',
 'teamSize': 15,
 'attacksPerMember': 2,
 'battleModifier': 'none',
 'preparationStartTime': '20240912T155551.000Z',
 'startTime': '20240913T160943.000Z',
 'endTime': '20240914T160943.000Z',
 'clan': {'tag': '#2GLLPQYLL',
  'name': 'Espookys',
  'badgeUrls': {'small': 'https: //api-assets.clashofclans.com/badges/70/4vLvAkaNNp2h-f8z58GVtDIJj0P5F9GliQV3ov25krg.png',
   'large': 'https: //api-assets.clashofclans.com/badges/512/4vLvAkaNNp2h-f8z58GVtDIJj0P5F9GliQV3ov25krg.png',
   'medium': 'https: //api-assets.clashofclans.com/badges/200/4vLvAkaNNp2h-f8z58GVtDIJj0P5F9GliQV3ov25krg.png'},
  'clanLevel': 7,
  'attacks': 17,
  'stars': 32,
  'destructionPercentage': 80.6,
  'members': [{'tag': '#UGRL8LV8',
    'name': 'mcnasty',
    'townhallLevel': 13,
    'mapPosition': 5,
    'attacks': [{'attackerTag': '#UGRL8LV8',
      'defenderTag': '#P2JGVYRP',
      'stars': 3,
      'destructionPercentage': 100,
      'order': 28,
      'duration': 75},
     {'attackerTag': '#U

In [39]:
data = json_normalize(response, record_path=['clan', 'members'])
data

,tag,name,townhallLevel,mapPosition,attacks,opponentAttacks,bestOpponentAttack.attackerTag,bestOpponentAttack.defenderTag,bestOpponentAttack.stars,bestOpponentAttack.destructionPercentage,bestOpponentAttack.order,bestOpponentAttack.duration
0,#UGRL8LV8,mcnasty,13,5,"[{'attackerTag': '#UGRL8LV8', 'defenderTag': '...",2,#2JGVRGVRV,#UGRL8LV8,2,93,14,173
1,#8Q8UQL8Y,NMbaseball22,11,9,NaN,1,#GL2GC2PQ8,#8Q8UQL8Y,3,100,8,150
2,#8CRG2JR9,MeekMulli,12,6,"[{'attackerTag': '#8CRG2JR9', 'defenderTag': '...",1,#2JGVRGVRV,#8CRG2JR9,3,100,15,127
3,#G222UU009,Ben Dover,11,12,"[{'attackerTag': '#G222UU009', 'defenderTag': ...",1,#GY9CPCLC8,#G222UU009,3,100,19,144
4,#899PLPG,cordell,14,4,NaN,1,#Y9CJRGJYC,#899PLPG,3,100,5,122
5,#2RR2PVRG,Alwayz High,15,2,NaN,1,#9R8CYRJP9,#2RR2PVRG,3,100,3,145
6,#9PRCGP9Y,Luke,15,3,"[{'attackerTag': '#9PRCGP9Y', 'defenderTag': '...",1,#Y9CJRGJYC,#9PRCGP9Y,2,99,4,180
7,#9GGVU8C8,Shamrock 2.0,16,1,"[{'attackerTag': '#9GGVU8C8', 'defenderTag': '...",1,#9R8CYRJP9,#9GGVU8C8,2,73,22,147
8,#L99GG88LG,BigAedo,12,8,NaN,1,#9VC2P0CQR,#L99GG88LG,3,100,17,130
9,#GYQ09LYRR,winston,12,7,"[{'attackerTag': '#GYQ09LYRR', 'defenderTag': ...",1,#9VC2P0CQR,#GYQ09LYRR,3,100,16,91


In [40]:
members_df = json_normalize(response, record_path=['clan', 'members'], meta=[])
total_members_df = members_df.copy()


In [41]:
try:
    attacks_df = json_normalize(
        members_df.to_dict(orient='records'),  # Convert DataFrame to dicts
        record_path='attacks',
        meta=['name'],
        meta_prefix='member_'
    )
    attacks_df
except KeyError:
    attacks_df = pd.DataFrame(columns=[
    'attackerTag', 'defenderTag', 'stars', 'destructionPercentage',
    'order', 'duration', 'member_name'
    ])

In [42]:
attacks_df['attack_num'] = attacks_df.groupby('member_name').cumcount() + 1
attacks_df



,attackerTag,defenderTag,stars,destructionPercentage,order,duration,member_name,attack_num
0,#UGRL8LV8,#P2JGVYRP,3,100,28,75,mcnasty,1
1,#UGRL8LV8,#LCGR0Y2RC,3,100,30,96,mcnasty,2
2,#8CRG2JR9,#LCGR0Y2RC,1,83,10,141,MeekMulli,1
3,#G222UU009,#G8JQCRRCR,3,100,20,118,Ben Dover,1
4,#G222UU009,#GL2GC2PQ8,1,76,32,113,Ben Dover,2
5,#9PRCGP9Y,#82CY8YUUJ,3,100,31,112,Luke,1
6,#9PRCGP9Y,#9VC2P0CQR,3,100,33,110,Luke,2
7,#9GGVU8C8,#9R8CYRJP9,1,77,21,109,Shamrock 2.0,1
8,#9GGVU8C8,#Y9CJRGJYC,2,97,24,180,Shamrock 2.0,2
9,#GYQ09LYRR,#LV2990URP,3,100,12,124,winston,1


In [43]:
attacks_df['total_attacks'] = attacks_df.groupby('member_name')['attack_num'].transform('max')
attacks_df

,attackerTag,defenderTag,stars,destructionPercentage,order,duration,member_name,attack_num,total_attacks
0,#UGRL8LV8,#P2JGVYRP,3,100,28,75,mcnasty,1,2
1,#UGRL8LV8,#LCGR0Y2RC,3,100,30,96,mcnasty,2,2
2,#8CRG2JR9,#LCGR0Y2RC,1,83,10,141,MeekMulli,1,1
3,#G222UU009,#G8JQCRRCR,3,100,20,118,Ben Dover,1,2
4,#G222UU009,#GL2GC2PQ8,1,76,32,113,Ben Dover,2,2
5,#9PRCGP9Y,#82CY8YUUJ,3,100,31,112,Luke,1,2
6,#9PRCGP9Y,#9VC2P0CQR,3,100,33,110,Luke,2,2
7,#9GGVU8C8,#9R8CYRJP9,1,77,21,109,Shamrock 2.0,1,2
8,#9GGVU8C8,#Y9CJRGJYC,2,97,24,180,Shamrock 2.0,2,2
9,#GYQ09LYRR,#LV2990URP,3,100,12,124,winston,1,2


In [44]:
total_attacks_df = attacks_df[['member_name', 'total_attacks']].drop_duplicates().set_index('member_name')
total_attacks_df

,total_attacks
member_name,
mcnasty,2
MeekMulli,1
Ben Dover,2
Luke,2
Shamrock 2.0,2
winston,2
Tony,2
Drippy,1
MewThree,1


In [45]:
pivot_df = attacks_df.pivot_table(
    index=['member_name'], 
    columns='attack_num', 
    values=['attackerTag', 'defenderTag', 'stars', 'destructionPercentage', 'order', 'duration'], 
    aggfunc='first'
)
pivot_df

attackerTag             defenderTag              \
attack_num              1           2           1           2   
member_name                                                     
Ben Dover      #G222UU009  #G222UU009  #G8JQCRRCR  #GL2GC2PQ8   
Drippy         #LPG2LYYGJ         NaN  #GL2YCLVCC         NaN   
Luke            #9PRCGP9Y   #9PRCGP9Y  #82CY8YUUJ  #9VC2P0CQR   
MeekMulli       #8CRG2JR9         NaN  #LCGR0Y2RC         NaN   
MewThree       #GGV0JPQ2C         NaN  #GL2YCLVCC         NaN   
Shamrock 2.0    #9GGVU8C8   #9GGVU8C8  #9R8CYRJP9  #Y9CJRGJYC   
Tony           #GPC0JCLPG  #GPC0JCLPG  #LJVUCC9RC  #GLPJPGPQ2   
Wreckit Ralph  #GPRURPLRG  #GPRURPLRG  #GY9CPCLC8  #GLPJPGPQ2   
mcnasty         #UGRL8LV8   #UGRL8LV8   #P2JGVYRP  #LCGR0Y2RC   
winston        #GYQ09LYRR  #GYQ09LYRR  #LV2990URP  #GL2GC2PQ8   

              destructionPercentage        duration        order       stars  \
attack_num                        1      2        1      2     1     2     1   
member_name                                                                    
Ben Dover                     100.0   76.0    118.0  113.0  20.0  32.0   3.0   
Drippy                         45.0    NaN     92.0    NaN  11.0   NaN   1.0   
Luke                          100.0  100.0    112.0  110.0  31.0  33.0   3.0   
MeekMulli                      83.0    NaN    141.0    NaN  10.0   NaN   1.0   
MewThree                       47.0    NaN     93.0    NaN  25.0   NaN   1.0   
Shamrock 2.0                   77.0   97.0    109.0  180.0  21.0  24.0   1.0   
Tony                          100.0   65.0    114.0   89.0   6.0   7.0   3.0   
Wreckit Ralph                 100.0  100.0    102.0  132.0   2.0  29.0   3.0   
mcnasty                       100.0  100.0     75.0   96.0  28.0  30.0   3.0   
winston                       100.0   88.0    124.0   87.0  12.0  13.0   3.0   

                    
attack_num       2  
member_name         
Ben Dover      1.0  
Drippy         NaN  
Luke           3.0  
MeekMulli      NaN  
MewThree       NaN  
Shamrock 2.0   2.0  
Tony           2.0  
Wreckit Ralph  3.0  
mcnasty        3.0  
winston        1.0

In [46]:
pivot_df.columns = [f"{col[0]}_{col[1]}" for col in pivot_df.columns]

In [47]:
def extract_text_and_number(column_name):
    match = re.match(r'(.+?)_(\d+)', column_name)
    return (int(match.group(2)), match.group(1)) if match else (float('inf'), column_name)


In [48]:
sorted_columns = sorted(pivot_df.columns, key=lambda x: extract_text_and_number(x))
sorted_columns

['attackerTag_1',
 'defenderTag_1',
 'destructionPercentage_1',
 'duration_1',
 'order_1',
 'stars_1',
 'attackerTag_2',
 'defenderTag_2',
 'destructionPercentage_2',
 'duration_2',
 'order_2',
 'stars_2']

In [49]:
pivot_df = pivot_df[sorted_columns]

In [50]:
final_df = pivot_df.join(total_attacks_df)
final_df

,attackerTag_1,defenderTag_1,destructionPercentage_1,duration_1,order_1,stars_1,attackerTag_2,defenderTag_2,destructionPercentage_2,duration_2,order_2,stars_2,total_attacks
member_name,,,,,,,,,,,,,
Ben Dover,#G222UU009,#G8JQCRRCR,100.0,118.0,20.0,3.0,#G222UU009,#GL2GC2PQ8,76.0,113.0,32.0,1.0,2
Drippy,#LPG2LYYGJ,#GL2YCLVCC,45.0,92.0,11.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1
Luke,#9PRCGP9Y,#82CY8YUUJ,100.0,112.0,31.0,3.0,#9PRCGP9Y,#9VC2P0CQR,100.0,110.0,33.0,3.0,2
MeekMulli,#8CRG2JR9,#LCGR0Y2RC,83.0,141.0,10.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1
MewThree,#GGV0JPQ2C,#GL2YCLVCC,47.0,93.0,25.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1
Shamrock 2.0,#9GGVU8C8,#9R8CYRJP9,77.0,109.0,21.0,1.0,#9GGVU8C8,#Y9CJRGJYC,97.0,180.0,24.0,2.0,2
Tony,#GPC0JCLPG,#LJVUCC9RC,100.0,114.0,6.0,3.0,#GPC0JCLPG,#GLPJPGPQ2,65.0,89.0,7.0,2.0,2
Wreckit Ralph,#GPRURPLRG,#GY9CPCLC8,100.0,102.0,2.0,3.0,#GPRURPLRG,#GLPJPGPQ2,100.0,132.0,29.0,3.0,2
mcnasty,#UGRL8LV8,#P2JGVYRP,100.0,75.0,28.0,3.0,#UGRL8LV8,#LCGR0Y2RC,100.0,96.0,30.0,3.0,2


In [51]:
new_name_df = final_df
new_name_df['Avg %'] = new_name_df[['destructionPercentage_1', 'destructionPercentage_2']].mean(axis=1)
new_name_df['Total Stars'] = new_name_df[["stars_1", "stars_2"]].sum(axis=1)

In [52]:
new_order = ['total_attacks', 'Avg %', 'destructionPercentage_1', 'destructionPercentage_2', 
             'stars_1', 'stars_2', 'Total Stars']
new_name_df = new_name_df[new_order]
final_name_df = new_name_df.rename(columns={'destructionPercentage_1' : 'Attack 1 %', 'destructionPercentage_2' : 'Attack 2 %', 
                                            'stars_1' : 'Attack 1 Stars', 'stars_2' : 'Attack 2 Stars', 'total_attacks' : 'Total Attacks'})
final_name_df

,Total Attacks,Avg %,Attack 1 %,Attack 2 %,Attack 1 Stars,Attack 2 Stars,Total Stars
member_name,,,,,,,
Ben Dover,2,88.0,100.0,76.0,3.0,1.0,4.0
Drippy,1,45.0,45.0,NaN,1.0,NaN,1.0
Luke,2,100.0,100.0,100.0,3.0,3.0,6.0
MeekMulli,1,83.0,83.0,NaN,1.0,NaN,1.0
MewThree,1,47.0,47.0,NaN,1.0,NaN,1.0
Shamrock 2.0,2,87.0,77.0,97.0,1.0,2.0,3.0
Tony,2,82.5,100.0,65.0,3.0,2.0,5.0
Wreckit Ralph,2,100.0,100.0,100.0,3.0,3.0,6.0
mcnasty,2,100.0,100.0,100.0,3.0,3.0,6.0


In [53]:
try:
    total_members_df = total_members_df.set_index('name')
except KeyError:
    pass


In [54]:
combined_df = final_name_df.reindex(total_members_df.index)
combined_df.fillna(0, inplace=True)
combined_df.sort_values(["Total Attacks", 'Total Stars', 'Avg %'], ascending=False, inplace=True)

In [55]:
def colors(row):
    if row['Total Attacks'] == 2.0:
        # color = "rgb(0,255,0)" # For Online Viewer
        color = "green" # For Excel
    elif row['Total Attacks'] == 1.0:
        color = "yellow"
    else:
        color = "red"
    return [f'color: {color}'] * len(row) 

In [56]:
styled_df = combined_df.style.apply(colors, axis=1).format('{:.1f}')
new_file = file_name.replace('.json', '.xlsx')
styled_df.to_excel(f'./charts/{new_file}', engine='openpyxl', index=True)
# combined_df.to_csv('./charts/data.csv', index=True)
# combined_df.to_json('./charts/data.json', orient='records', lines=True, index=True)
# combined_df.to_json('./charts/datai.json', orient='index')
# combined_df.to_json('./charts/datas.json', orient='split')
# combined_df.to_json('./charts/datat.json', orient='table')
# combined_df.to_html('./charts/data.html', index=True)
styled_df


,Total Attacks,Avg %,Attack 1 %,Attack 2 %,Attack 1 Stars,Attack 2 Stars,Total Stars
name,,,,,,,
mcnasty,2.0,100.0,100.0,100.0,3.0,3.0,6.0
Luke,2.0,100.0,100.0,100.0,3.0,3.0,6.0
Wreckit Ralph,2.0,100.0,100.0,100.0,3.0,3.0,6.0
Tony,2.0,82.5,100.0,65.0,3.0,2.0,5.0
winston,2.0,94.0,100.0,88.0,3.0,1.0,4.0
Ben Dover,2.0,88.0,100.0,76.0,3.0,1.0,4.0
Shamrock 2.0,2.0,87.0,77.0,97.0,1.0,2.0,3.0
MeekMulli,1.0,83.0,83.0,0.0,1.0,0.0,1.0
MewThree,1.0,47.0,47.0,0.0,1.0,0.0,1.0
